# Analyzing Online Ticket Sales with Amazon Redshift

In this workspace, I will be accessing data stored in Amazon Redshift, a data warehouse product that is part of Amazon Web Services. More specifically, I'll be analyzing sales activity from a fictional ticketing website where users both buy and sell tickets online for sporting events, shows, and concerts ([source](https://docs.aws.amazon.com/redshift/latest/dg/c_sampledb.html)).


## Explore events

In [1]:
-- List of all events
Select * from event

,eventid,venueid,catid,dateid,eventname,starttime
0,6649,6,9,1827,Hannah Montana,2008-01-01 19:30:00+00:00
1,1433,248,6,1827,Grease,2008-01-01 19:00:00+00:00
2,4135,16,9,1827,Nas,2008-01-01 14:30:00+00:00
3,5807,45,9,1827,Return To Forever,2008-01-01 15:00:00+00:00
4,1217,238,6,1827,Mamma Mia!,2008-01-01 20:00:00+00:00
...,...,...,...,...,...,...
8793,6034,45,9,2191,War,2008-12-31 14:00:00+00:00
8794,6783,60,9,2191,The Police,2008-12-31 15:00:00+00:00
8795,6857,18,9,2191,Judas Priest,2008-12-31 14:00:00+00:00
8796,7192,54,9,2191,Lindsey Buckingham,2008-12-31 19:30:00+00:00


This is linking up to several other tables in the warehouse, such as venue, category and date. Let's join things up.

In [1]:
SELECT *
FROM event
INNER JOIN venue USING(venueid)
INNER JOIN category USING(catid)
INNER JOIN date USING(dateid)
LIMIT 100

,dateid,catid,venueid,eventid,eventname,starttime,venuename,venuecity,venuestate,venueseats,catgroup,catname,catdesc,caldate,day,week,month,qtr,year,holiday
0,1827,8,303,423,La Damnation de Faust,2008-01-01 19:30:00+00:00,Grand 1894 Opera House,Galveston,TX,0.0,Shows,Opera,All opera and light opera,2008-01-01 00:00:00+00:00,WE,1,JAN,1,2008,True
1,1827,7,211,2099,The Bacchae,2008-01-01 19:30:00+00:00,The Broadway Theatre,New York City,NY,0.0,Shows,Plays,All non-musical theatre,2008-01-01 00:00:00+00:00,WE,1,JAN,1,2008,True
2,1827,9,38,6640,Herbie Hancock,2008-01-01 19:30:00+00:00,US Airways Center,Phoenix,AZ,0.0,Concerts,Pop,All rock and pop music concerts,2008-01-01 00:00:00+00:00,WE,1,JAN,1,2008,True
3,1827,9,9,8621,Hot Chip,2008-01-01 19:00:00+00:00,Dick's Sporting Goods Park,Commerce City,CO,0.0,Concerts,Pop,All rock and pop music concerts,2008-01-01 00:00:00+00:00,WE,1,JAN,1,2008,True
4,1827,6,235,1376,The King and I,2008-01-01 14:30:00+00:00,Studio 54,New York City,NY,0.0,Shows,Musicals,Musical theatre,2008-01-01 00:00:00+00:00,WE,1,JAN,1,2008,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1827,9,6,6649,Hannah Montana,2008-01-01 19:30:00+00:00,New York Giants Stadium,East Rutherford,NJ,80242.0,Concerts,Pop,All rock and pop music concerts,2008-01-01 00:00:00+00:00,WE,1,JAN,1,2008,True
96,1827,6,248,1433,Grease,2008-01-01 19:00:00+00:00,Charles Playhouse,Boston,MA,0.0,Shows,Musicals,Musical theatre,2008-01-01 00:00:00+00:00,WE,1,JAN,1,2008,True
97,1827,9,16,4135,Nas,2008-01-01 14:30:00+00:00,TD Banknorth Garden,Boston,MA,0.0,Concerts,Pop,All rock and pop music concerts,2008-01-01 00:00:00+00:00,WE,1,JAN,1,2008,True
98,1827,9,45,5807,Return To Forever,2008-01-01 15:00:00+00:00,Prudential Center,Newark,NJ,0.0,Concerts,Pop,All rock and pop music concerts,2008-01-01 00:00:00+00:00,WE,1,JAN,1,2008,True


There's a `starttime` column coming from the `event` table and there's also a `caldate` column, coming from the `date` table. Let's see what's up with this.

In [2]:
SELECT
	CASE WHEN date(caldate)=date(starttime) THEN true ELSE false END AS same_date,
	count(*)
FROM event
INNER JOIN date USING(dateid)
GROUP BY 1

,same_date,count
0,True,8095
1,False,703


In [3]:
SELECT max(datediff('hour', caldate, starttime))
FROM event
INNER JOIN date USING(dateid)

,max
0,20


Let's see how much events are happening in different cities.

In [4]:
SELECT 
	venuecity,
    count(*) AS num_event
FROM event
INNER JOIN venue USING(venueid)
GROUP BY 1
ORDER BY 2 DESC

,venuecity,num_event
0,New York City,2647
1,Los Angeles,312
2,Las Vegas,300
3,Chicago,209
4,San Francisco,194
...,...,...
74,Newark,27
75,Montreal,27
76,Irving,25
77,Sunrise,24


## Explore listings and sales

In [5]:
select * from listing limit 100
-- show 100 listing records

,listid,sellerid,eventid,dateid,numtickets,priceperticket,totalprice,listtime
0,2092,42560,8609,1827,22,194.0,4268.0,2008-01-01 05:49:06+00:00
1,614,25339,770,1827,10,236.0,2360.0,2008-01-01 05:07:30+00:00
2,9242,31448,7441,1827,14,88.0,1232.0,2008-01-01 02:39:54+00:00
3,776,20797,1811,1827,18,133.0,2394.0,2008-01-01 06:59:39+00:00
4,9730,8250,2772,1827,20,146.0,2920.0,2008-01-01 09:46:40+00:00
...,...,...,...,...,...,...,...,...
95,86956,6313,4731,1827,2,83.0,166.0,2008-01-01 04:07:20+00:00
96,105898,33212,7332,1827,1,256.0,256.0,2008-01-01 10:36:07+00:00
97,89786,19102,7476,1827,18,354.0,6372.0,2008-01-01 02:42:47+00:00
98,114948,6324,5300,1827,18,223.0,4014.0,2008-01-01 09:30:14+00:00


In [6]:
select * from sales limit 100
-- show 100 sales records

,salesid,listid,sellerid,buyerid,eventid,dateid,qtysold,pricepaid,commission,saletime
0,33095,36572,30047,660,2903,1827,2,234.0,35.10,2008-01-01 09:41:06+00:00
1,88268,100813,45818,698,8649,1827,4,836.0,125.40,2008-01-01 07:26:20+00:00
2,150314,173969,48680,816,8762,1827,2,688.0,103.20,2008-01-01 03:50:02+00:00
3,110917,127048,37631,116,1749,1827,1,337.0,50.55,2008-01-01 07:05:02+00:00
4,157751,206999,3003,157,6605,1827,1,1730.0,259.50,2008-01-01 12:50:55+00:00
...,...,...,...,...,...,...,...,...,...,...
95,40196,44927,32034,477,5931,1831,4,432.0,64.80,2008-01-05 03:41:37+00:00
96,50877,57387,37390,572,8538,1831,1,233.0,34.95,2008-01-05 01:56:15+00:00
97,50084,56553,23982,3808,1104,1831,1,245.0,36.75,2008-01-05 11:32:27+00:00
98,60724,68779,37824,1850,7626,1831,1,487.0,73.05,2008-01-05 03:23:28+00:00


Seeing if multiple sales can happen for the same listing.

In [7]:
WITH listings_with_sales AS (
    SELECT 
        listid,
        count(*) AS number_of_sales
    FROM listing
	INNER JOIN sales USING(listid)
    GROUP BY 1
)
SELECT
	number_of_sales,
    count(*) as cont
FROM listings_with_sales
GROUP BY 1
ORDER BY 1

,number_of_sales,cont
0,1,48029
1,2,36570
2,3,14665
3,4,1808
4,5,12


It turns out the bulk of listings have only one sale associated with them. There are 12 listings that had 5 sales.

 To find the user that sold the most tickets in 2008:

In [8]:
SELECT
	sellerid,
    username,
    (firstname ||' '|| lastname) as name,
	city,
    sum(qtysold)
FROM sales
INNER JOIN date USING(dateid)
INNER JOIN users ON sales.sellerid = users.userid
WHERE year=2008
GROUP BY 1, 2, 3, 4
ORDER BY 5 desc
LIMIT 5

,sellerid,username,name,city,sum
0,48950,TUT90BHI,Nayda Hood,Frisco,46
1,19123,DZW00VOQ,Scott Simmons,Carson,41
2,20029,RPM45HGY,Drew Mcguire,Lancaster,41
3,36791,DCE77DOA,Emerson Delacruz,Springfield,40
4,9697,GDM25KSM,Dorian Ray,Vicksburg,39


Similarly, to find the most active buyer on the site in 2008:

In [9]:
select
	buyerid,
    username,
    (firstname ||' '|| lastname) as name,
	city,
    sum(qtysold)
FROM sales
INNER JOIN date USING(dateid)
INNER JOIN users ON sales.buyerid = users.userid
WHERE year = 2008
GROUP BY 1, 2, 3, 4
ORDER BY 5 desc
LIMIT 5

,buyerid,username,name,city,sum
0,8933,CNF70VPH,Jerry Nichols,Middlebury,67
1,1298,EDB46JXK,Kameko Bowman,Newburyport,64
2,3797,KTV94TWB,Armando Lopez,Pomona,64
3,5002,CBC51API,Kellie Savage,Falls Church,63
4,3881,XJN46RCL,Herrod Sparks,Rome,60


Let's see if there's a big difference in average sales price for different categories of events. I'm looking at actual sales here, not listings!

In [ ]:
SELECT
	avg(pricepaid/qtysold) as avgticketprice
    catgroup
FROM sales
INNER JOIN event using(eventid)
INNER JOIN category using(catid)
GROUP BY 1

Error: SELECT
	avg(pricepaid/qtysold) as avgticketprice
    catgroup
FROM sales
INNER JOIN event using(eventid)
INNER JOIN category using(catid)
GROUP BY 1 - syntax error at or near "catgroup"

Are there listings where the sale happened before the listing?

In [10]:
SELECT COUNT(*)
FROM listing
INNER JOIN sales using(listid)
WHERE listtime>saletime

,count
0,2965


This must be bad data! Let's keep these out when figuring out the shortest and longest time to get a listing sold.

In [ ]:
WITH tts AS (
    SELECT datediff('seconds', listtime, saletime) as time_to_sale
    FROM sales
    INNER JOIN listing USING(listid)
    WHERE listtime < saletime
)
SELECT 
	min(time_to_sell) as shortest_inseconds,
    max(time_to_sell)/3600/24 as longest_indays
FROM tts

Error: WITH tts AS (
    SELECT datediff('seconds', listtime, saletime) as time_to_sale
    FROM sales
    INNER JOIN listing USING(listid)
    WHERE listtime < saletime
)
SELECT 
	min(time_to_sell) as shortest_inseconds,
    max(time_to_sell)/3600/24 as longest_indays
FROM tts - column "time_to_sell" does not exist in tts

## Finding users that should advertise

Suppose the owners of the ticketing website want to target certain users with the suggestion to advertise their listings, so they have a higher chance of actually getting sales. To do this, build a list of users that had the most outstanding listings in terms of price per ticket.

In [11]:
WITH listings_with_sales AS (
    SELECT 
        listid,
 	    listing.sellerid,
        numtickets AS tickets_listed,
        priceperticket,
		sum(coalesce(qtysold, 0)) as tickets_sold
    FROM listing
	LEFT JOIN sales USING(listid)
    GROUP BY 1, 2, 3, 4
)
SELECT 
	sellerid,
    (firstname ||' '|| lastname) as name,
	sum((tickets_listed-tickets_sold)*priceperticket) AS unrealized_sales
FROM listings_with_sales lws
INNER JOIN users ON lws.sellerid = users.userid
GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 100

,sellerid,name,unrealized_sales
0,25428,Jaime Wagner,58395.0
1,24896,Macey Ortiz,53086.0
2,49322,Dustin Vincent,50914.0
3,36926,Audrey Barber,50345.0
4,45819,Kelly Barrett,49826.0
...,...,...,...
95,35926,Ulysses Kinney,38895.0
96,45372,Lysandra Sanchez,38862.0
97,25373,Jakeem Byrd,38847.0
98,48188,Caesar Parrish,38847.0


Looks like Jaime Wagner had 58k of unrealized sales!

## Bonus: visualize sales over time

In [12]:
-- Show total pricepaid on a weekly basis--
select date_trunc('week', saletime) as sales_week,
sum(pricepaid) as total_sales
from sales
group by 1
order by 1

,sales_week,total_sales
0,2007-12-31 00:00:00+00:00,244991.0
1,2008-01-07 00:00:00+00:00,734374.0
2,2008-01-14 00:00:00+00:00,1206758.0
3,2008-01-21 00:00:00+00:00,1613119.0
4,2008-01-28 00:00:00+00:00,1875270.0
5,2008-02-04 00:00:00+00:00,2013802.0
6,2008-02-11 00:00:00+00:00,2400791.0
7,2008-02-18 00:00:00+00:00,2481096.0
8,2008-02-25 00:00:00+00:00,2080490.0
9,2008-03-03 00:00:00+00:00,2419824.0


In [ ]:
import plotly.express as px
___